In [ ]:
# streamlit code to api call for all coins to get historical data and save to sql 

In [ ]:
# Import the required libraries and dependencies
import streamlit as st
import pandas as pd
import numpy as np
import requests
import json
import sqlalchemy as sql
import datetime
%matplotlib inline



coin_list = ["bitcoin", "ethereum", "tether", "binancecoin", "usd-coin", "binance-usd", "ripple", "cardano", "dogecoin", "matic-network"]
coin_id = st.sidebar.selectbox(label="Select Coin of Interest: ", coin_list)

def get_historical_data ():
    coin_list = ["bitcoin", "ethereum", "tether", "binancecoin", "usd-coin", "binance-usd", "ripple", "cardano", "dogecoin", "matic-network"]
    start_time = 1641042000 # end time 2022-01-01 1pmGMT
    end_time = 1668208735 #now
    try:
    for c in coin_list:
        url = f"https://api.coingecko.com/api/v3/coins/{c}/market_chart/range?vs_currency=usd&from={start_time}&to={end_time}"
        response = requests.get(url)
        data = response.json()
        timestamp_list = []
        price_list = []
        for row in data['prices'] :
            timestamp_list.append(datetime.datetime.fromtimestamp(row[0]/1000))
            price_list.append(row[1])
    
        raw_data = {'timestamp': timestamp_list, 'price' : price_list}
        df = pd.DataFrame(raw_data, columns=['timestamp','price'])
        df = df.set_index('timestamp')
        pd.options.display.float_format = '{:.2f}'.format
        return df
    except json.JSONDecodeError:
        pass
    finally:
        return df

days = 90
def indicators (days):
   #plot of pice 
   df.plot(figsize=(10,7),title="price of bitcoin by day over last 90days")
#SMA - Simple Moving Average with plot
df["SMA7"] = df["price"].rolling(7).mean()
df.dropna(inplace=True)

df["SMA30"] = df["price"].rolling(30).mean()
df.dropna(inplace=True)

df["SMA90"] = df["price"].rolling(90).mean()
df.dropna(inplace=True)

df.plot(figsize=(10,7), title=" simple moving Average")

#SMA - Simple Moving Average with plot
df["SMA7"] = df["price"].rolling(7).mean()
df.dropna(inplace=True)

df["SMA30"] = df["price"].rolling(30).mean()
df.dropna(inplace=True)

df["SMA90"] = df["price"].rolling(90).mean()
df.dropna(inplace=True)

df.plot(figsize=(10,7), title=" simple moving Average")
   st.write("Notification !!!")

   
st.set_page_config(page_title="Notification", page_icon="")
st.markdown("# Notification Info")
st.sidebar.header("Notification Info")
st.write(
    """Notification information """
)

Notification()


In [ ]:
import pandas as pd

# function cleans all_coin_info_df and saves it to a database or csv?
def clean_coin_data (all_coiN_info_df):
    all_coin_info_df = all_coiN_info_df.drop(['image', 'fully_diluted_valuation',
                                 'ath', 'ath_change_percentage', 'ath_date',
                                 'atl', 'atl_change_percentage', 'atl_date', 'roi'], axis=1)
    all_coin_info_df = all_coin_info_df.sort_values(by='market_cap_rank', ascending=True)
    all_coin_info_df = all_coin_info_df.rename(columns = {'index' : 'id'})
    all_coin_info_df.set_index('id', inplace = True)
    all_coin_info_df["last_updated"] = pd.to_datetime(all_coin_info_df["last_updated"])
        
    return all_coin_info_df

In [ ]:
from pathlib import Path
import sqlalchemy as sql

#function saves all_coin_info_df to database, mabye add save to csv?
def save_coin_data (all_coin_info_df):
    database_connection_string = 'sqlite:///' #../Resources/all_coin_info.db'
    engine = sql.create_engine(database_connection_string, echo=True)
    # Create a database connection string
    all_coin_info_df.to_sql('coin_info', engine, index=False, if_exists='append')
    